In [1]:
import cv2
from ultralytics import YOLO

# Load the YOLO model
model = YOLO('yolov10m.pt')

# Open the webcam
cap = cv2.VideoCapture(0)  # 0 is usually the default webcam

# Define the scale factor for resizing the result frame
scale_factor = 1.5  # Increase this value to make the frame larger

# Define the size of the result frame
result_frame_size = (800, 600)

# Initialize variables for tracking
person_count = 0
previous_person_count = 0
movement_direction = ""

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO inference on the frame
    results = model(frame)

    # Initialize person count
    person_count = 0

    # Process the results
    for r in results:
        boxes = r.boxes
        for box in boxes:
            c = box.cls
            class_name = model.names[int(c)]
            
            if class_name == 'person':
                person_count += 1
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Detect movement direction
    if person_count > previous_person_count:
        movement_direction = "Entering"
    elif person_count < previous_person_count:
        movement_direction = "Exiting"
    else:
        movement_direction = ""

    # Create a new frame for the result
    result_frame = cv2.resize(frame, result_frame_size)

    # Add person count and movement direction to the result frame
    cv2.putText(result_frame, f'Total Persons: {person_count}', (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(result_frame, f'Movement: {movement_direction}', (10, 60), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

    # Display the result frame
    cv2.imshow("RESULT", result_frame)

    # Update the previous person count
    previous_person_count = person_count

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'cv2'